Before we begin, let us execute the below cell to display information about the NVIDIA® CUDA® driver and the GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by clicking on it with your mouse, and pressing Ctrl+Enter, or pressing the play button in the toolbar above. You should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:

- Learn how to run the same code on both a multicore CPU and a GPU using standard language parallelism
- Understand steps required to make a sequential code parallel using ISO Fortran and ISO C++

We do not intend to cover:
- Detailed optimization techniques and mapping of standard constructs to CUDA C


<details>
    <summary markdown="span"><b>ISO C++</b></summary>
    
# Standard Template Library (STL)
    
If you are not familiar with STL (Standard Template Library), this section will give you a brief introduction that would be required to understand the usage of STL library for our code.

The C++ STL (Standard Template Library) is a powerful set of C++ template classes to provide general-purpose classes and functions with templates that implement many popular and commonly used algorithms and data structures like vectors, lists, queues, and stacks.

At the core of the C++ Standard Template Library are following three well-structured components 

- Containers: Containers are used to manage collections of objects of a certain kind. There are several different types of containers like dequeue, list, vector, map etc.

- Algorithms: Algorithms act on containers. They provide the means by which you will perform initialization, sorting, searching, and transforming of the contents of containers.

- Iterators: Iterators are used to step through the elements of collections of objects. These collections may be containers or subsets of containers.

For our code to make *Pair Calculation* we will be making use of ```vector``` container. The example below will introduce you to the container and how to use an iterator to step through elements of a vector. ```vector``` container (a C++ Standard Template) which is similar to an array with an exception that it automatically handles its own storage requirements in case it grows

For our code, we will be making use of ```std::for_each``` algorithm and its sample usage is also shown in the code below:

```cpp
#include <vector>
#include <algorithm>
#include <iostream>
 
//Using functor
struct Sum
{
    void operator()(int n) { sum += n; }
    int sum{0};
};
 
int main()
{
    std::vector<int> nums{3, 4, 2, 8, 15, 267};
 
    auto print = [](const int& n) { std::cout << " " << n; };
 
    std::cout << "before:";
    std::for_each(nums.cbegin(), nums.cend(), print);
    std::cout << '\n';
 
    std::for_each(nums.begin(), nums.end(), [](int &n){ n++; });
 
    // calls Sum::operator() for each number
    Sum s = std::for_each(nums.begin(), nums.end(), Sum());
 
    std::cout << "after: ";
    std::for_each(nums.cbegin(), nums.cend(), print);
    std::cout << '\n';
    std::cout << "sum: " << s.sum << '\n';
}
```

To learn more about STL you can read and execute sample codes [here](https://www.tutorialspoint.com/cplusplus/cpp_stl_tutorial.htm).


# Parallel STL
Starting with C++17, parallelism has become an integral part of the standard itself. Parallel STL is an implementation of the C++ standard library algorithms with support for execution policies, commonly called C++17.

C++17 Parallel Standard Library (stdpar) introduces parallel and vector concurrency for standard algorithms. It is important to note that stdpar is a library and not a language extension.


## `std::par` Execution Policies


Execution Policies define the kind of parallelism that will be applied to parallel algorithms. Most standard algorithms included in STL support execution policies. Defined below are the execution policies:

- `std::execution::seq` = sequential
    - This execution policy type is used as a unique type to disambiguate parallel algorithm overloading and requires that a parallel algorithm’s execution may not be parallelized.
- `std::execution::par` = parallel
    - This execution policy type is used as a unique type to disambiguate parallel algorithm overloading and indicate that a parallel algorithm’s execution may be parallelized
- `std::execution::par_unseq` = parallel + vectorized
    - This execution policy type used as a unique type to disambiguate parallel algorithm overloading and indicate that a parallel algorithm’s execution may be parallelized and vectorized

Implementation of execution policies is provided by different compilers from specific vendors. For GPU parallel execution policy we will be making use of the NVIDIA compiler. 


## Historical Perspective

Changes to how the call to _stl_ algorithms changed the new version of C++ standard to incorporate execution policies:

**C++98:** 
```cpp
std::sort(c.begin(), c.end()); 
```
**C++17:** 
```cpp
std::sort(std::execution::par, c.begin(), c.end());
```

We will be using the NVIDIA HPC C++ compiler, NVC++. It supports C++17, C++ Standard Parallelism (stdpar) for NVIDIA GPUs, OpenACC for multicore CPUs and NVIDIA GPUs, and OpenMP for multicore CPUs. No language extensions or non-standard libraries are required to enable GPU acceleration. All data movement between host memory and GPU device memory is performed implicitly and automatically under the control of CUDA [Unified Memory](../GPU_Architecture_Terminologies.ipynb), which means that heap memory is automatically shared between a CPU(Host) and GPU(Device). Stack memory and global memory are not shared. Below given example shows the right allocation and usage of the stdpar.

```cpp
std::vector<int> v = ...;
std::sort(std::execution::par, v.begin(), v.end()); // OK, vector allocates on heap

std::array<int, 1024> a = ...;
std::sort(std::execution::par, a.begin(), a.end()); // Fails, array stored on the stack
```

For our code we will be making use of ```std::for_each``` algorithm with support for ```std::execution::par``` execution policy

**Counting Iterator**: In our code we will also be using a special iterator ```counting_iterator```. This iterator  represents a pointer into a range of sequentially changing values. This iterator is useful for creating a range filled with a sequence without explicitly storing it in memory. Using ```counting_iterator``` saves memory capacity and bandwidth
</details>
<br/>
    
<details>
    <summary markdown="span"><b>ISO Fortran</b></summary>
    
# Fortran Standard Parallelism

ISO Standard Fortran 2008 introduced the DO CONCURRENT construct to allow you to express loop-level parallelism, one of the various mechanisms for expressing parallelism directly in the Fortran language. 

Fortran developers have  been able to accelerate their programs using CUDA Fortran, OpenACC or OpenMP. Now with the support of DO CONCURRENT on GPU with NVIDIA HPC SDK, the compiler automatically accelerates loops using DO CONCURRENT, allowing developers to get the benefit of accelerating  on NVIDIA GPUs using ISO Standard Fortran without any extensions, directives, or non-standard libraries. You can now write standard Fortran, remaining fully portable to other compilers and systems, and still benefit from the full power of NVIDIA GPUs

For our code to make *Pair Calculation* all that’s required is expressing loops with DO CONCURRENT. The example below will introduce you to the syntax of DO CONCURRENT 

Sample vector addition code is shown in code below:

```fortran
  subroutine vec_addition(x,y,n)
    real :: x(:), y(:)
    integer :: n, i  
    do i = 1, n 
      y(i) = x(i)+y(i)
    enddo  
  end subroutine vec_addition
```

In order to make use of ISO Fortran DO CONCURRENT we need to replace the `do` loop with `do concurrent` as shown in code below

```fortran
  subroutine vec_addition(x,y,n)
    real :: x(:), y(:)
    integer :: n, i  
    do concurrent (i = 1: n) 
      y(i) = x(i)+y(i)
    enddo  
  end subroutine vec_addition
```

By changing the DO loop to DO CONCURRENT, you are telling the compiler that there are no data dependencies between the n loop iterations. This leaves the compiler free to generate instructions that the iterations can be executed in any order and simultaneously. The compiler parallelizes the loop even if there are data dependencies, resulting in race conditions and likely incorrect results. It’s your responsibility to ensure that the loop is safe to be parallelized.

## Nested Loop Parallelism

Nested loops are a common code pattern encountered in HPC applications. A simple example might look like the following:

```fortran
  do i=2, n-1
    do j=2, m-1
      a(i,j) = w0 * b(i,j) 
    enddo
  enddo
```

It is straightforward to write such patterns with a single DO CONCURRENT statement, as in the following example. It is easier to read, and the compiler has more information available for optimization.

```fortran
  do concurrent(i=2 : n-1, j=2 : m-1)
    a(i,j) = w0 * b(i,j) 
  enddo
```
</details>
<br/>


Now, lets start <mark>modifying the original code and add the necessary changes to parallelise the code. Without changing the orginal code, you will get error running the below cells.</mark> Click on the <b>[C++ version](../source_code/rdf.cpp)</b> or the <b>[Fortran version](../source_code/rdf.f90)</b> links, and start modifying the C or Fortran version of the RDF code. Remember to **SAVE** your code after changes, before running below cells.

### Compile and Run for Multicore

Now, let's compile the code. We will be using NVIDIA HPC SDK for this exercise. The flags used for enabling standard parallelism for target offloading are as follows:

- `-stdpar` : This flag enables standard parallelism for the target architecture
- `-stdpar=multicore` will allow us to compile our code for a multicore
- `-stdpar` will allow us to compile our code for a NVIDIA GPU (Default is NVIDIA)

After running the cells, you can inspect part of the compiler feedback for C++ or Fortran version and see what it's telling us (your compiler feedback will be similar to the below).

### <mark>Compile the code for multicore (C++)</mark>

In [ ]:
#Compile the code for multicore (C++)
!cd ../source_code && echo "compiling C++ version .. " && nvc++ -std=c++17 -stdpar=multicore -Minfo \
-I/opt/nvidia/hpc_sdk/Linux_x86_64/23.5/cuda/11.8/include \
-o rdf_c rdf.cpp -fopenmp \
-L/opt/nvidia/hpc_sdk/Linux_x86_64/23.5/cuda/11.8/lib64 -lnvToolsExt && ./rdf_c && cat Pair_entropy.dat

The output should be the following:

    
```
    
s2 value is -2.43191
s2bond value is -3.87014
    
```
    
and the compiler feedback would look similar as below:
    
```
main:
     79, stdpar: Generating Multicore code
         79, std::fill with std::execution::par policy parallelized on CPU
pair_gpu(double *, double *, double *, std::atomic<int> *, int, int, double, double, double, int):
    191, stdpar: Generating Multicore code
        191, std::for_each with std::execution::par policy parallelized on CPU
main:
    117, FMA (fused multiply-add) instruction(s) generated
    146, FMA (fused multiply-add) instruction(s) generated
    147, FMA (fused multiply-add) instruction(s) generated
```

In [ ]:
#profile and see output of nvptx (C++ version)
!cd ../source_code && nsys profile -t nvtx --stats=true --force-overwrite true -o rdf_stdpar_multicore ./rdf_c

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> the [C++ version](../source_code/rdf_stdpar_multicore.nsys-rep) then choosing <mark>save Link As</mark> Once done, open it via the GUI and have a look at the example expected profiler report below:

**Example screenshot (C++ code)**

<img src="../../_common/images/stdpar_multicore.png">


### <mark>Compile the code for multicore (Fortran)</mark>

In [ ]:
#Compile the code for multicore (Fortran)
!cd ../source_code && echo "compiling Fortran version .. " && nvfortran -stdpar=multicore -Minfo -o rdf_f rdf.f90 -lnvhpcwrapnvtx && ./rdf_f && cat Pair_entropy.dat

**Note:** Since we are targeting the NVTX v3 API, a header-only C library, and added Fortran-callable wrappers to the code, we add `-lnvhpcwrapnvtx` at the compile time to do the link to the library.

The output should be the following:

```
    
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527 
    
```
and the compiler feedback would look similar as below:
    
```
rdf:
     80, Memory zero idiom, loop replaced by call to __c_mzero8
     92, Generating Multicore code
         92, Loop parallelized across CPU threads
```

In [ ]:
#profile and see output of nvptx (Fortran version)
!cd ../source_code && nsys profile -t nvtx --stats=true --force-overwrite true -o rdf_doconcurrent_multicore ./rdf_f

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> the  [Fortran version](../source_code/rdf_doconcurrent_multicore.nsys-rep) then choosing <mark>save Link As</mark> Once done, open it via the GUI and have a look at the example expected profiler report below:

**Example screenshot (Fortran code)**
    
<img src="../../_common/images/do_concurrent_multicore.jpg">


### Compile and run for NVIDIA GPU

Without changing the code now let us try to recompile the code for NVIDIA GPU and rerun. GPU acceleration of standard parallel algorithms is enabled with the `-⁠stdpar` command-line option when using NVIDIA HPC Fortran or C++ compiler. If `-⁠stdpar `is specified, almost all algorithms that use a parallel execution policy are compiled for offloading to run in parallel on an NVIDIA GPU.

**Understand and analyze** the solutions for [C++](../source_code/SOLUTION/rdf.cpp) and [Fortran](../source_code/SOLUTION/rdf.f90) versions and compare with your versions. Once done, compile your code by running below cells.

Make sure to validate the output by running the executable and validate the output.

### <mark>Compile for Tesla GPU (ISO C++)</mark>

In [ ]:
#compile for Tesla GPU (ISO C++)
!cd ../source_code && echo "compiling C++ version .. " && nvc++ -std=c++17 -DUSE_COUNTING_ITERATOR  -stdpar=gpu -Minfo -o rdf_c rdf.cpp 


The output should be the following:
   
```
    
s2 value is -2.43191
s2bond value is -3.87014
    
```
    
and the compiler feedback would look similar as below:
    
```
main:
     79, stdpar: Generating NVIDIA GPU code
         79, std::fill with std::execution::par policy parallelized on GPU
pair_gpu(double *, double *, double *, std::atomic<int> *, int, int, double, double, double, int):
    188, stdpar: Generating NVIDIA GPU code
        188, std::for_each with std::execution::par policy parallelized on GPU
main:
     16, include "nvToolsExt.h"
        1494, include "nvtxImpl.h"
              119, FMA (fused multiply-add) instruction(s) generated
              148, FMA (fused multiply-add) instruction(s) generated
              149, FMA (fused multiply-add) instruction(s) generated
pair_gpu(double *, double *, double *, std::atomic<int> *, int, int, double, double, double, int)::[lambda(unsigned int) (instance 1)]::operator ()(unsigned int) const:
     16, include "nvToolsExt.h"
        1494, include "nvtxImpl.h"
              200, FMA (fused multiply-add) instruction(s) generated
              201, FMA (fused multiply-add) instruction(s) generated
              202, FMA (fused multiply-add) instruction(s) generated
              204, FMA (fused multiply-add) instruction(s) generated
```


In [ ]:
#profile and see output of nvptx (ISO C++ version)
!cd ../source_code && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_stdpar_gpu ./rdf_c

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> the [C++ version](../source_code/rdf_stdpar_gpu.nsys-rep) then choosing <mark>save Link As</mark> Once done, open it via the GUI and have a look at the example expected profiler report below:

**Example screenshot (ISO C++ code)**

<img src="../../_common/images/stdpar_gpu.png">

If you inspect the output of the profiler closer, you can see the *Unified Memory* usage. Moreover, if you compare the NVTX marker `Pair_Calculation` (from the NVTX row) in both multicore and GPU version, you can see how much improvement you achieved. 

Feel free to checkout the solution for [ISO C++](../source_code/SOLUTION/rdf.cpp) version to help you understand better.

### <mark>Compile for Tesla GPU (ISO Fortran)</mark>

In [ ]:
#compile for Tesla GPU (ISO Fortran)
!cd ../source_code &&  echo "compiling Fortran version .. "  && nvfortran -stdpar=gpu -Minfo -acc -o rdf_f rdf.f90 -lnvhpcwrapnvtx

**Note:** Since we are targeting the NVTX v3 API, a header-only C library, and added Fortran-callable wrappers to the code, we add `-lnvhpcwrapnvtx` at the compile time to do the link to the library.

The output should be the following:

```
    
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527 
    
```
and the compiler feedback would look similar as below:
    
```rdf:
     80, Memory zero idiom, loop replaced by call to __c_mzero8
     92, Generating Tesla code
         92, Loop parallelized across CUDA thread blocks, CUDA threads(4) blockidx%y threadidx%y
             Loop parallelized across CUDA thread blocks, CUDA threads(32) ! blockidx%x threadidx%x
```


In [ ]:
#profile and see output of nvptx (ISO Fortran version)
!cd ../source_code && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_doconcurrent_gpu ./rdf_f

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> the  [Fortran version](../source_code/rdf_doconcurrent_gpu.nsys-rep) then choosing <mark>save Link As</mark> Once done, open it via the GUI and have a look at the example expected profiler report below:

**Example screenshot (ISO Fortran code)**
    
<img src="../../_common/images/do_concurrent_gpu.jpg">


If you inspect the output of the profiler closer, you can see the *Unified Memory* usage. Moreover, if you compare the NVTX marker `Pair_Calculation` (from the NVTX row) in both multicore and GPU version, you can see how much improvement you achieved. 

Feel free to checkout the solutions for [ISO Fortran](../source_code/SOLUTION/rdf.f90) version to help you understand better.

# stdpar Analysis

**Usage Scenarios**
- stdpar is part of the standard language, all C++ compilers are expected to support it moving forward. This is C++ standard-compliant code, so you can maintain a single codebase. It provides a good start for accelerating code on accelerators like GPU and multicores.
- DO-CONCURRENT is part of the standard language and provides a good start for accelerating code on accelerators like GPU and multicores.

**Limitations/Constraints**
1. This isn’t a catch-all solution or necessarily an alternative to other programming models that provide more control over things like thread management. *std:par* and *DO CONCURRENT* provide the highest portability and can be seen as the first step to porting on GPU. The general abstraction limits the optimization functionalities. For example, the implementations are currently dependent on Unified memory. Moreover, one does not have control over thread management and that will limit performance improvement.
2. C++ constructs can only be used in the code using C++17 features and may not work for legacy codes.

**Which Compilers Support stdpar on GPUs and Multicore?**
1. NVIDIA GPU: As of Jan 2021, the HPC SDK compiler from NVIDIA supports std::par and DO-CONCURRENT on NVIDIA GPU.
2. x86 Multicore: 
    - stdpar: GCC has an implementation on a multicore CPU which is based on Intel TBB in the backend
    - DO CONCURRENT: Other compilers like intel compiler have an implementation on a multicore CPU

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommended you go to your browser's file menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip file of the files you have been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f _files.zip
zip -r _files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> [Here](../_files.zip) then choosing <mark>save Link As</mark>.

<!--Let us now go back to parallelizing our code using other approaches.

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

-----

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start.ipynb>HOME</a></p>

-----
-->

# Links and Resources
[Blog post on Developing Accelerated Code with Standard Language Parallelism](https://developer.nvidia.com/blog/developing-accelerated-code-with-standard-language-parallelism/)

[Blog post on Accelerating Standard C++ with GPUs Using stdpar](https://developer.nvidia.com/blog/accelerating-standard-c-with-gpus-using-stdpar/)

[Blog post on Accelerating Fortran DO CONCURRENT with GPUs and the NVIDIA HPC SDK](https://developer.nvidia.com/blog/accelerating-fortran-do-concurrent-with-gpus-and-the-nvidia-hpc-sdk/)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)


**NOTE**: To be able to see the Nsight Systems profiler output, please download the latest version of Nsight Systems from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.